<a href="https://colab.research.google.com/github/jammy0903/study/blob/main/AI_sec_1weekHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torchvision.models as models

import matplotlib.pyplot as plt

from torchvision import datasets, transforms
from torch.utils.data import Dataset, ConcatDataset, DataLoader

def load_CIFAR10(root_dataset, batch_size, width=32, height=32):
    image_transform = transforms.Compose([
            transforms.Resize((width, height)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # CIFAR-10 정규화
    ])

    train_cifar10 = datasets.CIFAR10(root=root_dataset, train=True, download=True, transform=image_transform)
    test_cifar10 = datasets.CIFAR10(root=root_dataset, train=False, download=True, transform=image_transform)

    train_cifar10_loader = DataLoader(dataset=train_cifar10, batch_size=batch_size, shuffle=True)
    test_cifar10_loader = DataLoader(dataset=test_cifar10, batch_size=batch_size, shuffle=False)

    nb_classes_cifar10 = len(test_cifar10.classes)

    print(f'\nNumber of Train Data\t: {len(train_cifar10_loader.dataset)}')
    print(f'Number of Test Data\t: {len(test_cifar10_loader.dataset)}')
    print(f'Number of Classes\t: {nb_classes_cifar10}')

    return train_cifar10_loader, test_cifar10_loader, nb_classes_cifar10



In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.models as models
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt

# CIFAR-10 데이터 로드 함수
def load_CIFAR10(root_dataset, batch_size, width=32, height=32):
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    transform_test = transforms.Compose([
        transforms.Resize((width, height)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    # CIFAR-10 데이터셋 다운로드 및 로드
    train_cifar10 = datasets.CIFAR10(root=root_dataset, train=True, download=True, transform=transform_train)
    test_cifar10 = datasets.CIFAR10(root=root_dataset, train=False, download=True, transform=transform_test)

    train_loader = DataLoader(dataset=train_cifar10, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_cifar10, batch_size=batch_size, shuffle=False)

    nb_classes = len(test_cifar10.classes)

    return train_loader, test_loader, nb_classes

# ResNet-18 모델 불러오기 함수
def get_resnet18(num_classes):
    model = models.resnet18(pretrained=False)  # pretrained=False로 설정하여 처음부터 학습
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

# 트레이닝 함수
def train(device, model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0

    for x, y in train_loader:
        optimizer.zero_grad()
        x = x.to(device).float()
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    return train_loss

# 검증 함수
def validate(device, model, test_loader, criterion):
    model.eval()
    test_loss = 0
    test_acc = 0

    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device).float()
            y = y.to(device)

            output = model(x)
            loss = criterion(output, y)
            _, pred = torch.max(output, 1)

            test_loss += loss.item()
            test_acc += torch.sum(pred == y.data).item()

    test_loss /= len(test_loader)
    test_acc /= len(test_loader.dataset)
    return test_loss, test_acc

if __name__ == '__main__':
    # 장치 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 하이퍼파라미터 설정
    root_dataset = './cifar10'
    batch_size = 64
    learning_rate = 0.01  # 초기 학습률
    epochs = 50  # 에폭 수
    momentum = 0.9
    weight_decay = 5e-4

    # CIFAR-10 데이터 로드
    train_loader, test_loader, nb_classes = load_CIFAR10(root_dataset, batch_size)

    # 모델 불러오기
    model = get_resnet18(nb_classes).to(device)

    # 손실 함수와 옵티마이저 설정
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

    # 학습률 스케줄러 설정 (7 에폭마다 학습률을 0.1배로 감소)
    scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

    # 학습 로그 저장용
    train_log = {
        'train_loss': [],
        'test_loss': [],
        'acc': []
    }

    # 학습 과정
    for epoch in range(epochs):
        train_loss = train(device, model, train_loader, criterion, optimizer)
        test_loss, test_acc = validate(device, model, test_loader, criterion)

        train_log['train_loss'].append(train_loss)
        train_log['test_loss'].append(test_loss)
        train_log['acc'].append(test_acc)

        # 학습률 스케줄러 업데이트
        scheduler.step()

        print(f'{epoch+1:5d}/{epochs:5d}: Train Loss [{train_loss:3.2f}] / Test Loss [{test_loss:3.2f}], Test Accuracy [{test_acc*100:3.2f}%]')

    # 학습 결과 그래프 출력
    plt.figure()
    plt.title('Train Loss vs Test Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.plot(train_log['train_loss'], label='Train')
    plt.plot(train_log['test_loss'], label='Test')
    plt.legend()
    plt.show()

    plt.figure()
    plt.title('Test Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.plot(train_log['acc'], label='Test')
    plt.legend()
    plt.show()


Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


    1/   50: Train Loss [1.70] / Test Loss [1.44], Test Accuracy [46.94%]
    2/   50: Train Loss [1.36] / Test Loss [1.18], Test Accuracy [59.36%]
    3/   50: Train Loss [1.17] / Test Loss [1.09], Test Accuracy [60.89%]
    4/   50: Train Loss [1.05] / Test Loss [0.94], Test Accuracy [66.71%]
    5/   50: Train Loss [0.96] / Test Loss [0.87], Test Accuracy [69.78%]
    6/   50: Train Loss [0.90] / Test Loss [0.80], Test Accuracy [71.93%]
    7/   50: Train Loss [0.83] / Test Loss [0.83], Test Accuracy [71.39%]
    8/   50: Train Loss [0.69] / Test Loss [0.66], Test Accuracy [76.87%]
    9/   50: Train Loss [0.65] / Test Loss [0.64], Test Accuracy [77.49%]
   10/   50: Train Loss [0.63] / Test Loss [0.64], Test Accuracy [77.94%]
   11/   50: Train Loss [0.63] / Test Loss [0.63], Test Accuracy [77.83%]
   12/   50: Train Loss [0.61] / Test Loss [0.62], Test Accuracy [78.18%]
   13/   50: Train Loss [0.60] / Test Loss [0.62], Test Accuracy [78.55%]
   14/   50: Train Loss [0.59] / Test 